In [9]:
#!/usr/bin/env python3
"""
Advanced QR Code Generator (fixed for Jupyter/Colab)

Default URL is set to:
https://www.youtube.com/watch?v=pvrkfb1C4tE&list=RD-_Av7re3Ce8&index=14

Usage examples (terminal):
  python qr_generator.py --url "https://example.com" --output my_qr.png
  python qr_generator.py --urls-file urls.txt --prefix batch_
"""

from __future__ import annotations
import subprocess
import sys
import os
import argparse

# -------------------------
# Optional dependency installer (best-effort)
# -------------------------
def install_package(package: str) -> None:
    """Install a package using pip (best-effort)."""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"{package} installed successfully.")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package}. Please install manually.")

# Try import qrcode and PIL, attempt to install if missing
try:
    import qrcode
    from PIL import Image
except Exception:
    print("Required packages not found. Attempting to install qrcode[pil] and pillow...")
    install_package("qrcode[pil]")
    install_package("pillow")
    try:
        import qrcode
        from PIL import Image
    except Exception as e:
        print(f"Import error after install attempt: {e}")
        print("Please install manually: pip install qrcode[pil] pillow")
        sys.exit(1)

# -------------------------
# QR generation helpers
# -------------------------
def _get_error_level(code: str):
    levels = {
        'L': qrcode.constants.ERROR_CORRECT_L,
        'M': qrcode.constants.ERROR_CORRECT_M,
        'Q': qrcode.constants.ERROR_CORRECT_Q,
        'H': qrcode.constants.ERROR_CORRECT_H,
    }
    return levels.get(code.upper())

def generate_qr_code(
    url: str,
    filename: str = "qr_code.png",
    size: int = 10,
    border: int = 4,
    error_correction: str = 'L',
    fill_color: str = "black",
    back_color: str = "white",
    verbose: bool = True
) -> bool:
    """
    Generate a QR code from the provided URL and save to filename.
    """
    try:
        err_level = _get_error_level(error_correction)
        if err_level is None:
            raise ValueError("Invalid error correction level. Choose from 'L','M','Q','H'.")

        qr = qrcode.QRCode(
            version=None,
            error_correction=err_level,
            box_size=int(size),
            border=int(border)
        )
        qr.add_data(url)
        qr.make(fit=True)

        img = qr.make_image(fill_color=fill_color, back_color=back_color)

        # Ensure PIL Image and save
        try:
            pil_img = img.convert("RGB")
        except Exception:
            pil_img = img  # already PIL image

        pil_img.save(filename)
        if verbose:
            print("QR code generated successfully.")
            print(f"Saved to: {filename}")
            print(f"URL encoded: {url}")
        return True

    except Exception as e:
        if verbose:
            print(f"Error generating QR: {e}")
        return False

def generate_qr_with_logo(
    url: str,
    logo_path: str,
    filename: str = "qr_code_with_logo.png",
    size: int = 10,
    border: int = 4,
    error_correction: str = 'H',
    fill_color: str = "black",
    back_color: str = "white",
    logo_scale: float = 0.2,
    verbose: bool = True
) -> bool:
    """
    Generate QR code and embed a centered logo.
    logo_scale is fraction of the QR's minimum side (e.g., 0.2 -> logo = 20%).
    """
    try:
        if not os.path.exists(logo_path):
            raise FileNotFoundError(f"Logo file not found: {logo_path}")

        err_level = _get_error_level(error_correction)
        if err_level is None:
            raise ValueError("Invalid error correction level. Choose from 'L','M','Q','H'.")

        qr = qrcode.QRCode(
            version=None,
            error_correction=err_level,
            box_size=int(size),
            border=int(border)
        )
        qr.add_data(url)
        qr.make(fit=True)

        qr_img = qr.make_image(fill_color=fill_color, back_color=back_color).convert("RGBA")

        logo = Image.open(logo_path).convert("RGBA")

        qr_w, qr_h = qr_img.size
        logo_size = int(min(qr_w, qr_h) * float(logo_scale))
        if logo_size <= 0:
            raise ValueError("Computed logo size <= 0. Check logo_scale value.")

        # Resampling compatibility across Pillow versions
        try:
            resample = Image.Resampling.LANCZOS  # Pillow >= 9.1.0
        except AttributeError:
            resample = Image.LANCZOS  # older Pillow

        logo = logo.resize((logo_size, logo_size), resample)

        pos = ((qr_w - logo_size) // 2, (qr_h - logo_size) // 2)
        qr_img.paste(logo, pos, logo)  # use logo as alpha mask

        final_img = qr_img.convert("RGB")
        final_img.save(filename)

        if verbose:
            print("QR code with logo generated successfully.")
            print(f"Saved to: {filename}")
            print(f"URL encoded: {url}")
        return True

    except Exception as e:
        if verbose:
            print(f"Error generating QR with logo: {e}")
        return False

def batch_generate_qr(
    urls_list,
    prefix: str = "qr_",
    size: int = 10,
    border: int = 4,
    error_correction: str = 'L',
    fill_color: str = "black",
    back_color: str = "white",
    verbose: bool = True
) -> int:
    success_count = 0
    for i, url in enumerate(urls_list, start=1):
        filename = f"{prefix}{i}.png"
        ok = generate_qr_code(url, filename, size, border, error_correction, fill_color, back_color, verbose)
        if ok:
            success_count += 1
        if verbose:
            print("-" * 40)
    if verbose:
        print(f"Total: {success_count}/{len(urls_list)} generated.")
    return success_count

# -------------------------
# Default URL (provided)
# -------------------------
DEFAULT_URL = "https://www.youtube.com/watch?v=pvrkfb1C4tE&list=RD-_Av7re3Ce8&index=14"

# -------------------------
# CLI entrypoint
# -------------------------
def main():
    parser = argparse.ArgumentParser(description="Advanced QR Code Generator")
    parser.add_argument("--url", type=str, default=DEFAULT_URL, help="URL to generate QR for (default: provided YouTube link)")
    parser.add_argument("--urls-file", type=str, help="Path to a file with URLs (one per line) for batch generation")
    parser.add_argument("--logo", type=str, help="Path to logo image to embed")
    parser.add_argument("--output", type=str, default="qr_code.png", help="Output filename")
    parser.add_argument("--size", type=int, default=10, help="Box size (default: 10)")
    parser.add_argument("--border", type=int, default=4, help="Border size (default: 4)")
    parser.add_argument("--error-correction", type=str, default='L', choices=['L','M','Q','H'], help="Error correction level")
    parser.add_argument("--fill-color", type=str, default="black", help="QR fill color")
    parser.add_argument("--back-color", type=str, default="white", help="QR background color")
    parser.add_argument("--logo-scale", type=float, default=0.2, help="Logo scale relative to QR min dimension (default 0.2)")
    parser.add_argument("--prefix", type=str, default="qr_", help="Prefix for batch files")
    parser.add_argument("--quiet", action="store_true", help="Suppress console messages")

    # Use parse_known_args so Jupyter/Colab injected args (e.g. -f /path/kernel-*.json) won't cause SystemExit
    args, unknown = parser.parse_known_args()
    verbose = not args.quiet

    if verbose and unknown:
        print("Note: ignored unknown args:", unknown)

    # Batch mode
    if args.urls_file:
        try:
            with open(args.urls_file, "r", encoding="utf-8") as f:
                urls = [line.strip() for line in f if line.strip()]
        except Exception as e:
            print(f"Error reading URLs file: {e}")
            sys.exit(1)
        if not urls:
            print("No URLs found in the provided file.")
            sys.exit(1)
        batch_generate_qr(urls, args.prefix, args.size, args.border, args.error_correction, args.fill_color, args.back_color, verbose)
        return

    # Single URL (uses default if none provided)
    if args.url:
        if args.logo:
            ok = generate_qr_with_logo(
                args.url,
                args.logo,
                args.output,
                args.size,
                args.border,
                args.error_correction,
                args.fill_color,
                args.back_color,
                args.logo_scale,
                verbose
            )
        else:
            ok = generate_qr_code(
                args.url,
                args.output,
                args.size,
                args.border,
                args.error_correction,
                args.fill_color,
                args.back_color,
                verbose
            )
        if not ok:
            sys.exit(1)
    else:
        # Should not happen because --url has a default, but kept for clarity
        print("No URL provided. Use --url or --urls-file.")
        parser.print_help()
        sys.exit(1)

if __name__ == "__main__":
    main()


Note: ignored unknown args: ['-f', '/root/.local/share/jupyter/runtime/kernel-8ff0fc53-04a6-404b-b39b-35f0af8b8c1a.json']
QR code generated successfully.
Saved to: qr_code.png
URL encoded: https://www.youtube.com/watch?v=pvrkfb1C4tE&list=RD-_Av7re3Ce8&index=14


In [ ]:
from google.colab import drive
drive.mount('/content/drive')